## Zooniverse Analysis

In [77]:
import json
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3
import numpy as np
import seaborn as sns

In [68]:
dfzoo = pd.read_csv('alert-production-zoo-classifications.csv')

In [69]:
len(dfzoo)

386

In [70]:
dfzoo.head()

,classification_id,user_name,user_id,user_ip,workflow_id,workflow_name,workflow_version,created_at,gold_standard,expert,metadata,annotations,subject_data,subject_ids
0,207552876,ebellm,1649740.0,bcb090b573bb93a473ab,13338,Scan cutouts,14.25,2020-03-07 01:13:05 UTC,NaN,NaN,"{""source"":""api"",""session"":""ef4df560cf0cb8504df...","[{""task"":""T0"",""task_label"":""How would you desc...","{""40938231"":{""retired"":null,""diaSourceId"":1764...",40938231
1,207552908,ebellm,1649740.0,bcb090b573bb93a473ab,13338,Scan cutouts,14.25,2020-03-07 01:13:13 UTC,NaN,NaN,"{""source"":""api"",""session"":""ef4df560cf0cb8504df...","[{""task"":""T0"",""task_label"":""How would you desc...","{""40938233"":{""retired"":null,""diaSourceId"":1764...",40938233
2,207553020,ebellm,1649740.0,bcb090b573bb93a473ab,13338,Scan cutouts,14.25,2020-03-07 01:13:44 UTC,NaN,NaN,"{""source"":""api"",""session"":""ef4df560cf0cb8504df...","[{""task"":""T0"",""task_label"":""How would you desc...","{""40938229"":{""retired"":null,""diaSourceId"":1764...",40938229
3,207553077,ebellm,1649740.0,bcb090b573bb93a473ab,13338,Scan cutouts,14.25,2020-03-07 01:14:00 UTC,NaN,NaN,"{""source"":""api"",""session"":""ef4df560cf0cb8504df...","[{""task"":""T0"",""task_label"":""How would you desc...","{""40938219"":{""retired"":null,""diaSourceId"":1764...",40938219
4,207553107,ebellm,1649740.0,bcb090b573bb93a473ab,13338,Scan cutouts,14.25,2020-03-07 01:14:09 UTC,NaN,NaN,"{""source"":""api"",""session"":""ef4df560cf0cb8504df...","[{""task"":""T0"",""task_label"":""How would you desc...","{""40938221"":{""retired"":null,""diaSourceId"":1764...",40938221


In [71]:
def get_classification(df):
    return dfzoo['annotations'].apply(json.loads).apply(lambda x: x[0]['value'])

def get_text_comment(df):
    return dfzoo['annotations'].apply(json.loads).apply(lambda x: x[1]['value'] if (x[0]['value'] == 'Other') else None)

def get_diaSourceId(df):
    return dfzoo['subject_data'].apply(json.loads).apply(lambda x: list(x.values())[0]['diaSourceId'])

In [72]:
dfzoo['classification'] = get_classification(dfzoo)
dfzoo['comment'] = get_text_comment(dfzoo)
dfzoo['diaSourceId'] = get_diaSourceId(dfzoo)

Foolishly we mixed HSC and DECam data, and for some reason it doesn't seem like the subject sets are made distinct.  I think I should have used some additional keys, not just the diaSourceID.

Thankfully there's no overlap in diaSourceId

In [63]:
REPO = '/project/mrawls/hits2015/rerun/decorr_2020_03'
connection = sqlite3.connect(f'{REPO}/association.db')
diaSources = pd.read_sql_query('select * from DiaSource', connection)

In [62]:
#REPO = '/project/mrawls/cosmos/rerun/ap_pipe-g-1'
#connection = sqlite3.connect(f'{REPO}/association.db')
#hscDiaSources = pd.read_sql_query('select * from DiaSource', connection)
#np.intersect1d(diaSources.diaSourceId.values,hscDiaSources.diaSourceId.values)

In [73]:
diaSources = pd.merge(diaSources,dfzoo[['classification','comment','diaSourceId']],on='diaSourceId',how='inner')

In [74]:
len(diaSources)

214

In [79]:
sns.barplot(data=diaSources,x='classification')

TypeError: unsupported operand type(s) for /: 'str' and 'int'